In [1]:
### code:

#### The code below defines a Problem class tailored for implementing a Sudoku game. 

#### Upon instantiation, the class receives the initial Sudoku board configuration as the starting state. Next, the actions method generates potential actions for the Sudoku game, corresponding to placing numbers (1-9) in empty cells on the grid.

#### After this, the result method is used, which is is pivotal for updating the Sudoku board following player moves. It aims to modify the state by placing the chosen number in an empty cell. 

The agent to be modelled is a Goal-Based Agent-
While we do not have a goal state, it is a goal based agent as our Sudoku-solving algorithm will prioritise decisions that bring us closer to achieving the goal of filling the entire puzzle grid with valid numbers. Each action taken by the algorithm will be aimed at reducing the distance from this goal, selecting moves that lead towards a solution state.
These agents rely on explicit knowledge representation, enabling them to make valid decisions and adapt their strategies as needed during the solving process to achieve the solution state of Sudoku puzzle.


In [2]:
import copy
from queue import Queue, PriorityQueue
import sys
from tkinter import *
class Problem(object):

    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        '''
        Return a list of possible actions
        '''
        return [x for x in range(1,len(state)+1)]

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        new_state=copy.deepcopy(state)
        for r in range(len(new_state)):
            for c in range(len(new_state[0])):
                if new_state[r][c] == 0:
                    new_state[r][c]= action
                    return new_state

#### Next, we use the 'goal_test' method to assess whether the current state of the Sudoku board meets the criteria for a goal state. A Sudoku puzzle is considered solved if the board is completely filled with numbers from 1 to 9, and there are no conflicts in rows, columns, or boxes (3x3 subgrids). If any of these conditions are not met, the method returns False, indicating that the puzzle is not yet solved.

#### The 'prune' method is used to evaluate whether the current state of the board can potentially lead to a solution. It checks for conflicts in rows, columns, and boxes, and if any conflict is found, it returns True, indicating that the current state is invalid and should be pruned from the search space.

#### The 'is_filled function' checks if the Sudoku board has any empty cells (marked with 0). If there are no empty cells, it returns True, indicating that the board is completely filled.

#### Lastly,  the 'check_rcs' method systematically scans each row and column of the Sudoku board. For each row or column, it maintains a history list to keep track of the numbers encountered. If a number is encountered more than once, indicating a duplicate within the same row or column, the method returns True, signifying a conflict. 

#### Otherwise, it continues scanning until all rows and columns have been checked. This mechanism ensures that each row and column adheres to the rules of Sudoku, preventing duplicate numbers within them.

In [3]:
    def goal_test(self, state):
        '''
        Check is the current state is the goal state. Goal state is when the 
        board is completely filled and there are no conflicts
        '''

        if not self.is_filled(state):
            return False

        if self.check_rcs(state):
            return False

        if self.check_boxes(state):
            return False
        return True

    def prune(self,state):
        '''
        This function will return True is the current state of the board 
        can never be part of a solution. Faslse otherwise.
        '''
        return (self.check_boxes(state) or self.check_rcs(state))

    def is_filled(self,state):
        '''
        This function returns true is the board has no blank cells. False otherwise.
        '''
        for row in state:
            if 0 in row:
                return False
        return True

    def check_rcs(self,state):
        ''' 
        This function checks all rows and columns to see if there are any conflicts.
        If there is a conflict, the function will return True. False otherwise.
        '''
                # Checks for rows
        for r in range(len(state)):
            histroy_list = [False]*10
            for c in range(len(state[0])):
                if histroy_list[state[r][c]]:
                    return True
                else:
                    if state[r][c]!=0:
                        histroy_list[state[r][c]] = True


        # Checks for cols
        for c in range(len(state[0])):
            histroy_list = [False]*10
            for r in range(len(state)):
                if histroy_list[state[r][c]]:
                    return True
                else:
                    if state[r][c]!=0:
                        histroy_list[state[r][c]] = True

        return False

    def check_boxes(self,state):

        '''
        This function checks sub boxes in the board for any conflicts. Returns True 
        if there is a conflict and false otherwise.
        '''
        board_len = len(state)
        box_len   = len(state)//3

        for colstart in range(0,board_len,3):
            for box in range(0,board_len-1,box_len):
                histroy_list = [False]*10
                for r in range (box,box+box_len):
                    for c in range(colstart,colstart+3):
                        if histroy_list[state[r][c]]:
                            return True
                        else:
                            if state[r][c]!=0:
                                histroy_list[state[r][c]] = True

        return False

#### The 'check_boxes' is used to examine each 3x3 subgrid (box) of the Sudoku board. It iterates through each box, maintaining a history list to track encountered numbers. 

#### If a duplicate number is found within any box, the function returns True, indicating a conflict. Otherwise, it continues scanning until all boxes have been checked. This process guarantees that each 3x3 subgrid complies with Sudoku rules, preventing duplicate numbers within them.

#### Node class instances are used to represent different board configurations encountered during the search for a solution. The state attribute stores the Sudoku board configuration associated with the node, while the parent attribute links to the node's parent in the search tree, facilitating backtracking. 

#### The action attribute records the action taken to transition from the parent node to the current node, in this context, it could be placing a number in a specific cell. The depth attribute indicates the depth of the node in the search tree, providing insight into the number of steps required to reach this state.

#### The expand method generates child nodes by applying possible actions to the current node's state, based on the Sudoku game's rules. Each child node is created by invoking the child_node method, which computes the next state and constructs a new Node instance.

#### Additionally, the '__lt__', '__eq__', and '__hash__' methods enable comparison and hashing of Node instances, facilitating operations like sorting and storing nodes in data structures such as priority queues.


In [4]:
class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        return Node(next_state, self, action)

    def __lt__(self, other):
        return self.depth < other.depth

    def __eq__(self, other):
        return self.depth == other.depth

    def __hash__(self):
        return hash(self.state)

    
 

#### The btr function implements a backtracking algorithm to solve Sudoku puzzles. It recursively explores possible solutions, returning the solution node or None if no solution exists.

#### It employs the is_safe function to ensure that placing a number in a cell follows Sudoku rules, checking for duplicates in rows, columns, and 3x3 subgrids.

#### In solve_sudoku, the algorithm iterates through empty cells, attempting numbers sequentially and recursively exploring each possibility until a valid solution is found or backtracking if no solution exists.

Uninformed Searches- Backtracking, bfs, dfs- not focusing on cost/heuristics

In [5]:
   
    
def btr(problem):
    """
    Solves a Sudoku problem using backtracking and recursion,
    returning the solution node or None if no solution exists.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        A Node object representing the solution or None if no solution exists.
    """

    def is_safe(board, row, col, num):
        # Check row and column for duplicates
        for i in range(len(board)):
            if board[row][i] == num or board[i][col] == num:
                return False

        # Check 3x3 subgrid for duplicates
        box_size = int(len(board) ** 0.5)
        box_row = row // box_size
        box_col = col // box_size
        for i in range(box_row * box_size, box_row * box_size + box_size):
            for j in range(box_col * box_size, box_col * box_size + box_size):
                if board[i][j] == num:
                    return False

        return True

    def solve_sudoku(board, row, col, parent):
        if row == len(board):
            return Node(board, parent)

        if col == len(board[0]):
            return solve_sudoku(board, row + 1, 0, parent)

        if board[row][col] != 0:
            return solve_sudoku(board, row, col + 1, parent)

        for num in range(1, len(board) + 1):
            if is_safe(board, row, col, num):
                new_board = copy.deepcopy(board)
                new_board[row][col] = num
                child = solve_sudoku(new_board, row, col + 1, Node(board, parent))
                if child is not None:
                    return child

        return None  # Backtrack

    initial_board = copy.deepcopy(problem.initial)
    solution = solve_sudoku(initial_board, 0, 0, None)
    if solution is not None:
        problem.initial = solution.state  # Update initial state with solution
        return solution
    else:
        return None



Solving Sudoku using Constraint Satisfaction Problem (CSP) and backtracking is both complete and optimal. Backtracking ensures completeness by systematically exploring all possible solutions until a valid one is found, while also guaranteeing an optimal solution by backtracking when encountering dead ends or invalid configurations. However, its efficiency can vary depending on the implementation and heuristics used. One way to increase efficiency is by employing constraint propagation techniques, such as forward checking or arc consistency, to reduce the search space by eliminating inconsistent values.

#### The breadth_first_search function orchestrates the exploration of a Sudoku puzzle's solution space using breadth-first search. It begins by creating a node representing the initial state. If the initial state fulfills the goal conditions, it returns the solution. Otherwise, it initializes a queue (frontier) to store nodes awaiting exploration.

#### Within the main loop, it dequeues nodes from the frontier, expands them, and checks if any child node satisfies the goal conditions. The expand method generates child nodes representing possible moves, and the goal_test method evaluates if a node meets the goal criteria.

#### Additionally, the function incorporates pruning via the prune method. If a child node is not None and passes the pruning condition, it's added to the frontier for further exploration.

In [6]:

def breadth_first_search(problem):
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node

    frontier=Queue()
    frontier.put(node)

    while frontier.qsize()!=0:
        node = frontier.get()
        problem.initial=node.state

        for child in node.expand(problem):
            if problem.goal_test(child.state):
                return child

            # Pruneing
            if child != None and problem.prune(child.state) is False:
                frontier.put(child)

    return None

Breadth first search algorithm is complete but is not optimal. BFS will always find a solution if one exists but since it does not take into account the cost of the path from the start state to the curent state, in other words, BFS will find the shortest from the start state to the goal state only if all edge costs are equal.

Ways to increase efficiency:

Implementing an iterative deepening search: An iterative deepening search limits the depth of the search at each iteration and gradually increases the depth limit. This technique ensures that BFS does not expand nodes beyond a certain depth, reducing the search space.


#### The depth_first_search function employs a depth-first search strategy to solve Sudoku puzzles. It begins by initializing a node representing the initial puzzle state and checks if it satisfies the goal conditions. If the initial state is the goal state, it returns the solution node.

#### The function utilizes a stack (frontier) to manage nodes for exploration. It iterates through nodes in the frontier, popping them for examination. If a node meets the goal conditions, it's returned as the solution. Otherwise, the function expands the node by generating child nodes representing possible moves.

#### Pruning is integrated to optimize the search process, discarding child nodes that do not meet the pruning condition specified by the prune method of the problem object.

In [7]:
def depth_first_search(problem):
    """
    Solves a Sudoku problem using depth-first search.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        A Node object representing the solution or None if no solution exists.
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node

    frontier = [node]

    while frontier:
        node = frontier.pop()

        if problem.goal_test(node.state):
            return node

        for child in node.expand(problem)[::-1]:
            if not problem.prune(child.state):
                frontier.append(child)

    return None

Depth First Search is not complete because the algorithm may get stuck in an infinite loop if there is a cycle in the graph. This means that our algorithm may not be able to find a path even if one exists.

In addition, it it not optimal because DFS does not guarantee that the found path is the shortest path.

Ways to increase efficiency:

implement backtracking to void exploring unnecessary paths -> this means that if the search reaches a dead end, we can backtrack to the last node with unexplored neighbors and continue the search from there.

#### Algorithms for UCS,Greedy, A*:

##### Greedy's Heuristics:
Minimum Remaining Values (MRV): This heuristic selects the variable (cell) with the fewest remaining legal values. It prioritizes the variables that are likely to lead to a solution faster.

##### Cost function for UCS:
the number of constraints violated as the cost function

#### The uniform_cost_search function solves Sudoku puzzles using Uniform Cost Search (UCS), prioritizing low-cost moves to minimize constraint violations. It initializes a node representing the initial puzzle state and checks if it meets the goal conditions. If the initial state is the goal state, it returns the solution node.

#### Utilizing a priority queue based on cost, the function explores nodes with lower-cost moves first. The cost for each child node is calculated using the calculate_cost function, which determines the number of constraints violated in the state.

#### The is_safe function plays a critical role by ensuring that placing a number in a cell adheres to Sudoku rules, checking for duplicates in rows, columns, and 3x3 subgrids.

In [8]:
def uniform_cost_search(problem):
    """
    Solves a Sudoku problem using Uniform Cost Search (UCS).

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        A Node object representing the solution or None if no solution exists.
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node

    frontier = PriorityQueue()  # Priority queue based on cost
    frontier.put((0, node))  # Initial cost is 0

    while not frontier.empty():
        cost, node = frontier.get()

        if problem.goal_test(node.state):
            return node

        for child in node.expand(problem):
            child_cost = calculate_cost(problem, child.state)
            frontier.put((child_cost, child))

    return None

def calculate_cost(problem, state):
    """
    Calculates the cost for a given Sudoku state based on the number of constraints violated.

    Args:
        problem: A Problem object representing the Sudoku puzzle.
        state: A 2D list representing the current state of the Sudoku puzzle.

    Returns:
        The cost of the state based on the number of constraints violated.
    """
    # Count the number of constraints violated
    constraints_violated = 0
    for r in range(len(state)):
        for c in range(len(state[0])):
            num = state[r][c]
            if num != 0:
                if not is_safe(state, r, c, num):
                    constraints_violated += 1

    return constraints_violated

def is_safe(state, row, col, num):
    """
    Checks if placing a number in a given cell is safe according to Sudoku rules.

    Args:
        state: A 2D list representing the current state of the Sudoku puzzle.
        row: The row index of the cell.
        col: The column index of the cell.
        num: The number to be placed in the cell.

    Returns:
        True if placing the number is safe, False otherwise.
    """
    # Check row and column for duplicates
    for i in range(len(state)):
        if state[row][i] == num or state[i][col] == num:
            return False

    # Check 3x3 subgrid for duplicates
    box_size = int(len(state) ** 0.5)
    box_row = row // box_size
    box_col = col // box_size
    for i in range(box_row * box_size, box_row * box_size + box_size):
        for j in range(box_col * box_size, box_col * box_size + box_size):
            if state[i][j] == num:
                return False

    return True


Using Uniform Cost Search (UCS) to solve Sudoku is both complete and optimal. UCS systematically explores the search space while considering the cost associated with each state, ensuring completeness by exhaustively searching for a solution and optimality by prioritizing nodes with lower costs.

However, the efficiency of UCS for solving Sudoku may be limited due to the large branching factor and the complexity of evaluating the cost function for each state. To enhance efficiency, heuristic techniques such as constraint propagation or domain reduction can be integrated to reduce the search space and guide the search towards promising solutions. 
Additionally, refining the cost function to better reflect the constraints violated in the Sudoku puzzle can improve the effectiveness of UCS in finding optimal solutions. By employing these strategies, UCS can efficiently navigate through the Sudoku solution space while guaranteeing completeness and optimality.

#### The solve_sudoku_greedy function employs a greedy search algorithm with the Minimum Remaining Values (MRV) heuristic to solve Sudoku puzzles. It iteratively selects empty cells with the fewest remaining legal values, prioritizing cells with the highest constraints.

#### The function traverses through the empty cells, selecting the one with the minimum remaining legal values. It then explores each legal value for the selected cell, trying them iteratively. If a value does not violate Sudoku rules, a new state is generated, and the function recursively calls itself with the updated problem state. This process continues until a solution is found or all legal values for the selected cell have been exhausted.

#### The count_legal_values function calculates the number of legal values for an empty cell, while get_legal_values determines the legal values for a given cell based on Sudoku rules.

In [9]:
def solve_sudoku_greedy(problem):
    """
    Solves a Sudoku puzzle using a greedy search algorithm with the Minimum Remaining Values (MRV) heuristic.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        The solution grid if a solution exists, or None if no solution is found.
    """
    grid = problem.initial
    empty_cells = [(row, col) for row in range(len(grid)) for col in range(len(grid[0])) if grid[row][col] == 0]

    while empty_cells:
        # Select the empty cell with the fewest remaining legal values (MRV heuristic)
        selected_cell = min(empty_cells, key=lambda cell: count_legal_values(grid, cell))

        # Choose a value for the selected cell that does not violate Sudoku rules
        legal_values = get_legal_values(grid, selected_cell)
        if not legal_values:
            return None  # Backtrack if no legal values are found

        # Try each legal value for the selected cell
        for value in legal_values:
            new_state = problem.result(grid, (selected_cell[0], selected_cell[1], value))
            new_problem = Problem(new_state)
            solution = solve_sudoku_greedy(new_problem)
            if solution:
                return solution  # Solution found

        # Remove the selected cell from the list of empty cells
        empty_cells.remove(selected_cell)

    return grid if problem.goal_test(grid) else None



def count_legal_values(grid, cell):
    """Counts the number of legal values for an empty cell."""
    return len(get_legal_values(grid, cell))

def get_legal_values(grid, cell):
    """Returns the legal values for an empty cell."""
    row, col = cell
    row_values = set(grid[row])
    col_values = set(grid[i][col] for i in range(len(grid)))  # Update range to len(grid)
    box_values = set(grid[i][j] for i in range(row//3*3, row//3*3 + 3)
                                    for j in range(col//3*3, col//3*3 + 3))
    return set(range(1, 10)) - row_values - col_values - box_values



Using a greedy search algorithm with the Minimum Remaining Values (MRV) heuristic to solve Sudoku is complete but not optimal. The algorithm iteratively selects empty cells with the fewest remaining legal values, prioritizing exploration of the most constrained cells first. 

While this approach guarantees completeness by exhaustively searching for a solution, it may not always find the most optimal solution as it doesn't consider the long-term consequences of its choices. Instead, it focuses on immediate local optimizations. 

Despite its lack of optimality, the greedy approach can efficiently navigate through the search space, especially in scenarios where the puzzle has a narrow solution space or where the MRV heuristic effectively guides the search towards promising solutions.

However, for more complex Sudoku puzzles with larger solution spaces, other search strategies such as backtracking or constraint satisfaction techniques may be more suitable for ensuring both completeness and optimality.

#### The solve_sudoku_a_star function addresses Sudoku puzzle solving using an A* search algorithm, amalgamating greedy search and uniform cost search techniques. It iteratively explores the Sudoku grid, prioritizing cells with the fewest remaining legal values (Minimum Remaining Values heuristic) similar to greedy search.

#### Within each iteration, the function selects an empty cell with the minimum remaining legal values and attempts to minimize constraint violations by choosing the value that incurs the lowest cost. It evaluates each legal value's cost based on the number of constraints violated using the calculate_cost function. The grid is then updated with the best value for the selected cell.

#### The function iterates through empty cells until a solution is found or no legal values are available for any cell. It returns the solution grid if a valid solution is discovered, or None if no solution exists.

In [10]:
from queue import PriorityQueue

def solve_sudoku_a_star(problem):
    """
    Solves a Sudoku puzzle using A* search combining greedy search and uniform cost search.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        The solution grid if a solution exists, or None if no solution is found.
    """
    grid = problem.initial
    empty_cells = [(row, col) for row in range(len(grid)) for col in range(len(grid[0])) if grid[row][col] == 0]

    while empty_cells:
        # Select the empty cell with the fewest remaining legal values (MRV heuristic)
        selected_cell = min(empty_cells, key=lambda cell: count_legal_values(grid, cell))

        # Choose a value for the selected cell that minimizes the number of constraints violated
        legal_values = get_legal_values(grid, selected_cell)
        if not legal_values:
            return None  # Backtrack if no legal values are found

        # Try each legal value for the selected cell
        min_cost = float('inf')
        best_value = None
        for value in legal_values:
            new_state = problem.result(grid, (selected_cell[0], selected_cell[1], value))
            new_problem = Problem(new_state)
            child_cost = calculate_cost(problem, new_state)
            if child_cost < min_cost:
                min_cost = child_cost
                best_value = value

        # Update the Sudoku grid with the best value for the selected cell
        grid[selected_cell[0]][selected_cell[1]] = best_value

        # Remove the selected cell from the list of empty cells
        empty_cells.remove(selected_cell)

    return grid if problem.goal_test(grid) else None


Solving Sudoku using A* search, which combines greedy search and uniform cost search, is complete and optimal under certain conditions. The algorithm iteratively selects empty cells with the fewest remaining legal values, guided by the Minimum Remaining Values (MRV) heuristic. It then chooses a value for the selected cell that minimizes the number of constraints violated, effectively balancing between local optimization and constraint satisfaction.

The completeness of the A* search is guaranteed as it exhaustively explores the search space until a solution is found. However, its optimality depends on the effectiveness of the heuristic used and the search strategy. In this case, the A* search prioritizes nodes with lower estimated costs, aiming to find the most promising solutions first. While it may not always guarantee the most optimal solution, it tends to converge towards it efficiently, especially in scenarios where the heuristic provides accurate estimates of the remaining costs.

By combining both greedy search and uniform cost search components, A* search can efficiently navigate through the Sudoku solution space while striving for optimality. However, its performance may vary depending on the heuristic quality and the complexity of the Sudoku puzzle. In cases where the heuristic is well-informed and the puzzle is not overly complex, A* search can provide both completeness and optimality in solving Sudoku puzzles.

#### The GUI class provides a graphical user interface for playing Sudoku games. It utilizes the Tkinter library to create a window with an adjustable grid size, allowing users to input values into cells and interact with various buttons for solving, clearing, and resizing the Sudoku board.

#### The create_board method sets up the Sudoku grid with Entry widgets for inputting values and buttons for solving, clearing, and resizing the board. The grid's background color is configured for visual appeal.

#### The get_values method retrieves the values inputted into the cells and returns them as a 2D array, where 0 represents empty cells. Conversely, the set_values method populates the GUI board with new values, replacing the existing ones.

#### Functionality to clear the board (clear), resize it (resize), and destroy all widgets (destroy_all) are also provided, ensuring flexibility and ease of use.

#### The solve method serves as the driver function, obtaining the Sudoku puzzle's initial state, solving it using various algorithms (such as backtracking, breadth-first search, depth-first search, uniform cost search, greedy search, and A* search), and displaying the solution on the GUI board if found. If no solution exists, it presents a message indicating the same.

In [11]:
from tkinter import messagebox

class  GUI:
    def __init__(self, n):
        self.board_size = n
        self.master = Tk()
        self.master.title("SUDOKU")
        self.create_board()

    def create_board(self):
        '''
        Funtion creates all text cells and buttons required
        '''
        self. cells = [[None for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col] = Entry(self.master, width=3)
                self.cells[row][col].grid(row=row,column=col)
                
                
        self.solve_button = Button(self.master, text="Solve", width=10, command=self.solve, bg="orange", fg="dark green")
        self.clear_button = Button(self.master, text="Clear", width=10, command=self.clear, bg="orange", fg="dark green")
        self.slider = Scale(self.master, from_=6, to=15, orient=HORIZONTAL, bg="purple", fg="white")
        self.resize_button = Button(self.master, text="Resize", width=10, command=lambda: self.resize(self.slider.get()), bg="orange", fg="dark green")

        # Configure background color for the board
        self.master.configure(bg="dark blue")

        self.solve_button.grid(row=0,column=self.board_size+1)
        self.clear_button.grid(row=1,column=self.board_size+1)
        self.slider.grid(row=2, column=self.board_size+1)
        self.resize_button.grid(row=3,column=self.board_size+1)

        self.controls = [self.solve_button,self.clear_button,self.resize_button,self.slider]

    def get_values(self):
        '''
        Returns values from the board as a 2 dimensional array. 0 represents empty cells
        '''
        values = [[0 for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                if self.cells[row][col].get():
                    values[row][col] = int(self.cells[row][col].get())
        return values

    def set_values(self, new_vals):
        '''
        Given a set of new values, function will write them on the GUI board. 
        Old values will be replaced.

        :param new_vals: (two-dimensional list) containing new values
        '''
        values = [[0 for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col].delete(0, END)
                self.cells[row][col].insert(0,new_vals[row][col])

    def clear(self):
        '''
        Clear values from all cells
        '''
        values = [[0 for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col].delete(0, END)

    def resize(self,val):
        '''
        Resize board to the value passed in.

        :param val: (int) size of board 
        '''
        self.destroy_all()
        self.board_size=val
        self.create_board()

    def destroy_all(self):
        '''
        Destroy all buttons,cells, and slider from the board
        '''
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col].destroy()

        for control in self.controls:
            control.destroy()


    def solve(self):
        '''
        Driver function
        '''
        values = self.get_values()
        p = Problem(values)
        solution = btr(p)
        #solution = breadth_first_search(p)
        #solution=depth_first_search(p)
        #solution=uniform_cost_search(p)
        #solution=solve_sudoku_greedy(p)
        #solution=solve_sudoku_a_star(p)
        if solution:
            self.set_values(solution.state)
        else:
            messagebox.showinfo("NA")

        


#### The main function initializes a 6x6 Sudoku board graphical interface using the GUI class and starts the main event loop using mainloop(). This allows users to interact with the Sudoku board, input values into cells, and utilize provided buttons for solving, clearing, and resizing the board. 

#### The function encapsulates the setup and event handling logic, making it convenient to run the Sudoku game application. It provides users with an intuitive platform to enjoy solving Sudoku puzzles, offering a visually pleasing and user-friendly environment for an engaging gameplay experience.

In [12]:
# using btr
def main():
    board = GUI(6)
    mainloop()

if __name__ == "__main__":
    main()

#### Screenshots of running this code with the UI can be seen in the readme file. The UCS and heuristic using functions take a lot of time to run since it doesn't make sense to use them to solve something like sudoku, however we still came up with algorithmic techniques to still implement them. 

# 